In [1]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [6]:
import subprocess
from pathlib import Path
from collections import Counter

In [5]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if True:
    cmd = [
        'python3', 'main.py',
        #'--api-base-url', 'http://localhost:8000/v1',
        '--api-key', 'xxx',
        '--model-name', 'unsloth/gemma-3-4b-it',
        '--chat-template-model-id', 'unsloth/gemma-3-4b-it',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

Running: python3 main.py --api-key xxx --model-name unsloth/gemma-3-4b-it --chat-template-model-id unsloth/gemma-3-4b-it --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 500 --prompt Write a story about a princess. 
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
--- Generation start ---

--- Generated Output (stream) ---


2025-05-18 17:11:19,326 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates survived even after relaxation.


Princess Aurelia of Silverwood wasn't known for her embroidery or her dancing. She wasn't known for her grace or her polite smiles. Aurelia was known for her mud. She was, quite simply, obsessed with mud. Not the refined, sculpted mud of a royal garden, but the glorious, glorious, dripping, squelching mud of the Silverwood Forest.

While other princesses her age practiced courtly manners and learned the names of rare flowers, Aurelia spent her days exploring the deepest, darkest corners of the forest, her boots caked in varying shades of ochre, grey, and deep, earthy black. The King, a weary man burdened by the weight of his crown and the expectation of a suitable bride, tolerated her eccentricities with a resigned shrug. Her mother, the Queen, simply worried.

“Aurelia, darling,” the Queen would say, smoothing a silk gown, “you’ll ruin your shoes. And your reputation!”

But Aurelia wasn’t interested in reputation. She was interested in the way mud felt between her toes, the way it sme

In [8]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

# Make sure you've launched the vllm openai-compatible server first. You can use something like:

# vllm serve unsloth/gemma-3-4b-it --port 8000 --max-model-len 2500 --served-model-name unsloth/gemma-3-4b-it --gpu-memory-utilization 0.95 --dtype bfloat16 --api-key xxx


THREADS  = 40
MAX_PROMPTS = 40

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8000/v1',
    '--api-key', 'xxx',
    #'--model-name', 'unsloth/gemma-3-4b-it',
    '--model-name', 'Qwen/Qwen3-4B',
    '--output-jsonl', OUT_JSONL,
    '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--request-mode', 'chunk',  # chunk / stream (stream mode might be buggy)
    '--chunk-size', '20', # how many tokens we request from the api before checking it for new violations
    '--chat-template-model-id', "Qwen/Qwen3-4B",
    '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-key xxx --model-name Qwen/Qwen3-4B --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 40 --max-prompts 40 --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 2000 --request-mode chunk --chunk-size 20 --chat-template-model-id Qwen/Qwen3-4B --tdpo-pairs-jsonl tdpo-pairs.jsonl
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
Extracting HF prompts:   0%|          | 39/177477 [00:00<00:04, 37355.07prompt/s]
Preparing to process 40 new prompts in this run.
BANNED (prompt_idx=4, type=ngram, ngram='one day')                         
BANNED (prompt_idx=5, type=slop_phrase, phrase='etched')                  
BANNED (prompt_idx=15, type=slop_phrase, phrase='scent')                  
BANNED (prompt_idx

2025-05-20 13:35:45,003 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.
2025-05-20 13:35:45,089 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.


BANNED (prompt_idx=14, type=slop_phrase, phrase='sunlight')              
BANNED (prompt_idx=25, type=slop_phrase, phrase='scent')                 
BANNED (prompt_idx=24, type=slop_phrase, phrase='flicker')               
BANNED (prompt_idx=26, type=slop_phrase, phrase='brow')                  
BANNED (prompt_idx=7, type=slop_phrase, phrase='whisper')                
BANNED (prompt_idx=12, type=slop_phrase, phrase='amusement')             
BANNED (prompt_idx=14, type=slop_phrase, phrase='subtle')                
BANNED (prompt_idx=23, type=slop_phrase, phrase='whisper')               
BANNED (prompt_idx=17, type=ngram, ngram='eyes wide')                    
BANNED (prompt_idx=20, type=ngram, ngram='first time')                   
BANNED (prompt_idx=2, type=slop_phrase, phrase='stretched')                      
BANNED (prompt_idx=31, type=slop_phrase, phrase='faint')                         
BANNED (prompt_idx=25, type=slop_phrase, phrase='rhythm')                        
BANNED (prompt

2025-05-20 13:35:46,681 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.
2025-05-20 13:35:46,712 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.
2025-05-20 13:35:46,742 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.


BANNED (prompt_idx=4, type=ngram, ngram='one day')                               
BANNED (prompt_idx=13, type=slop_phrase, phrase='scent')                         
BANNED (prompt_idx=14, type=ngram, ngram='something far')                        
BANNED (prompt_idx=24, type=slop_phrase, phrase='cracked')                       
BANNED (prompt_idx=16, type=slop_phrase, phrase='swirling')                      
BANNED (prompt_idx=6, type=slop_phrase, phrase='stared')                         
BANNED (prompt_idx=2, type=ngram, ngram='never seen')                            
BANNED (prompt_idx=12, type=slop_phrase, phrase='curiosity')                     
BANNED (prompt_idx=25, type=slop_phrase, phrase='warmth')                        
BANNED (prompt_idx=31, type=slop_phrase, phrase='glow')                          
BANNED (prompt_idx=10, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:(?!but\b|[.?!]).){1,50}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='not in a courtroom, but ')
BANNED (prom

...

2025-05-20 13:36:40,035 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.


BANNED (prompt_idx=4, type=slop_phrase, phrase='laced')                           
BANNED (prompt_idx=6, type=slop_phrase, phrase='whisper')                         
BANNED (prompt_idx=24, type=slop_phrase, phrase='footsteps')                      
BANNED (prompt_idx=7, type=slop_phrase, phrase='knew one thing')                  
BANNED (prompt_idx=39, type=slop_phrase, phrase='villain')                        
BANNED (prompt_idx=33, type=slop_phrase, phrase='muttered')                       
BANNED (prompt_idx=17, type=ngram, ngram='world around')                          
BANNED (prompt_idx=4, type=ngram, ngram='knew would')                             
BANNED (prompt_idx=38, type=slop_phrase, phrase='felt a chill run')               
BANNED (prompt_idx=22, type=slop_phrase, phrase='took a deep breath')             
BANNED (prompt_idx=16, type=slop_phrase, phrase='rhythm')                         
BANNED (prompt_idx=4, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:(?!but\b|[.

2025-05-20 13:37:06,349 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.


BANNED (prompt_idx=38, type=slop_phrase, phrase='smiled')                         
BANNED (prompt_idx=16, type=ngram, ngram='strange sense')                         
BANNED (prompt_idx=16, type=slop_phrase, phrase='took a deep breath')             
BANNED (prompt_idx=16, type=slop_phrase, phrase='knew one thing')                 
BANNED (prompt_idx=16, type=ngram, ngram='first time')                           
BANNED (prompt_idx=16, type=ngram, ngram='felt like')                            
BANNED (prompt_idx=16, type=ngram, ngram='felt like')                            
BANNED (prompt_idx=16, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:(?!but\b|[.?!]).){1,50}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='Not to be afraid, not to be lost, but ')
BANNED (prompt_idx=16, type=ngram, ngram='felt like')                            
BANNED (prompt_idx=16, type=slop_phrase, phrase='first time in a long')          
BANNED (prompt_idx=16, type=ngram, ngram='first time')                      

CompletedProcess(args=['python3', 'main.py', '--api-key', 'xxx', '--model-name', 'Qwen/Qwen3-4B', '--output-jsonl', 'creative_writing_generations.jsonl', '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT', '--hf-dataset-split', 'train', '--threads', '40', '--max-prompts', '40', '--logging-level', 'INFO', '--slop-phrases-file', 'banlists/slop_phrases.json', '--top-n-slop-phrases', '500', '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json', '--max-new-tokens', '2000', '--request-mode', 'chunk', '--chunk-size', '20', '--chat-template-model-id', 'Qwen/Qwen3-4B', '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'], returncode=0)